this is for when student selects only internships


In [54]:
import pandas as pd


In [55]:
students = pd.read_excel("studentsdataset.xlsx")
internship = pd.read_excel('internshipdataset.xlsx')
internship.drop(['Unnamed: 0','com_name.1'],axis=1,inplace=True)

In [56]:
internship.head(1)

,com_name,industry,job_title,job_description,prerequisites,location,is_startup,company_size,student_rank,salary
0,Tata Consultancy Services,Technology Services & Consulting,"Front-End Developer intern,Database Developer...",Join our team to develop scalable software sol...,"cpp,dsa,javascript,sql",Mumbai,0,large,all,15000


In [57]:
students.head(1)

,student_id,student_name,year,CGPA,skills,preferred_job_type,preferred_job_location,preferred_job_size,preferred_role,Category,expected_salary_pa,expected_salary_pm
0,232,Shubh Gupta,4,9.69,"cpp,python,dsa,ml,nlp,dl",internship,Mumbai,midsize,ml intern,advanced,0,68000.0


In [58]:
internship['prerequisites'] =internship['prerequisites'].str.replace(',',' ')
internship['prerequisites'] =internship['prerequisites'].str.lower()
internship['job_title'] =internship['job_title'].str.lower()
internship['job_description'] =internship['job_description'].str.lower()
internship['location'] =internship['location'].str.lower()
students['skills'] = students['skills'].str.replace(',',' ')


In [59]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### skills-prerequisites

In [60]:


lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills) 

students['lemmatized_skills'] = students['skills'].apply(lemmatize_skills)
internship['lemmatized_skills'] = internship['prerequisites'].apply(lemmatize_skills)

tfidf = TfidfVectorizer()
student_skill_vectors = tfidf.fit_transform(students['lemmatized_skills'])
internship_skill_vectors = tfidf.transform(internship['lemmatized_skills'])

similarity_score = cosine_similarity(student_skill_vectors,internship_skill_vectors)


def recommend_internships_skills_prerequisites(student_name_input):
    # student_name_input = input("Enter student name: ")
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()
        
    internship_indices = similarity_score[student_index].argsort()[::-1][:10]
    recommended_internship_skill = []
    weight_skill = 0.5
    for internship_index in internship_indices:
        # print(f"  - {company['com_name'][company_index]}")
        internship_name = internship['com_name'][internship_index]
        similarity = similarity_score[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_internship_skill.append((internship_name,weighted_similarity))
    recommended_internship_skill.sort(key=lambda x : x[1],reverse=True)    
    return recommended_internship_skill    



In [61]:
def recommend_students_skills(internship_name_input):
    try:
        internship_index = internship[internship['com_name'].str.lower() == internship_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{internship_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score[:, internship_index].argsort()[::-1][:19]
    recommended_students_skill = []
    weight_skill = 0.5
    for student_index in student_indices:
        # print(f"  - {students['student_name'][student_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_skill.append((student_name,weighted_similarity))
    recommended_students_skill.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_skill

### preferred role - job title

In [62]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills) 

students['lemmatized_preferred_role'] = students['preferred_role'].apply(lemmatize_skills)
internship['lemmatized_job_title'] = (internship['job_title'] + " " + internship['job_description']).apply(lemmatize_skills)

tfid = TfidfVectorizer()
students_preferred_role_vectors = tfid.fit_transform(students['lemmatized_preferred_role'])
internship_job_title_vectors = tfid.transform(internship['lemmatized_job_title'])

similarity_score_jobtitle_role = cosine_similarity(students_preferred_role_vectors,internship_job_title_vectors)


In [63]:
def recommend_internship_preferredrole_jobtitle(student_name_input):
    # student_name_input = input("Enter student name: ")
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    internship_indices = similarity_score_jobtitle_role[student_index].argsort()[::-1][:19]
    recommended_internship_jobtitle_role = []
    weight_skill = 0.6
    for internship_index in internship_indices:
        # print(f"  - {company['com_name'][company_index]}")
        internship_name = internship['com_name'][internship_index]
        similarity = similarity_score_jobtitle_role[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_internship_jobtitle_role.append((internship_name,weighted_similarity))
    recommended_internship_jobtitle_role.sort(key=lambda x : x[1],reverse=True)    
    return recommended_internship_jobtitle_role    

In [64]:
def recommend_students_preferredrole_jobtitle(internship_name_input):
    try:
        internship_index = internship[internship['com_name'].str.lower() == internship_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{internship_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_jobtitle_role[:, internship_index].argsort()[::-1][:10]
    recommended_students_jobtitle_role = []
    weight_skill = 0.6
    for student_index in student_indices:
        # print(f"  - {students['student_name'][student_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_jobtitle_role[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_jobtitle_role.append((student_name,weighted_similarity))
    recommended_students_jobtitle_role.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_jobtitle_role
        
  

### location-location

In [65]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills)

students['lemmatized_location'] = students['preferred_job_location'].apply(lemmatize_skills)
internship['lemmatized_location'] = internship['location'].apply(lemmatize_skills)

tfi = TfidfVectorizer()
students_location_vectors = tfi.fit_transform(students['lemmatized_location'])
internship_location_vectors = tfi.transform(internship['lemmatized_location'])

similarity_score_location = cosine_similarity(students_location_vectors,internship_location_vectors)

In [66]:
def recommend_internship_location_location(student_name_input):
    
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    internship_indices = similarity_score_location[student_index].argsort()[::-1][:19]
    recommended_internship_location = []
    weight_skill = 0.2
    for internship_index in internship_indices:
        # print(f"  - {company['com_name'][company_index]}")
        company_name = internship['com_name'][internship_index]
        similarity = similarity_score_location[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_internship_location.append((company_name,weighted_similarity))
    recommended_internship_location.sort(key=lambda x : x[1],reverse=True)    
    return recommended_internship_location 


In [67]:
def recommend_students_location_location(internship_name_input):
    try:
        internship_index = internship[internship['com_name'].str.lower() == internship_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{internship_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_location[:, internship_index].argsort()[::-1][:10]
    recommended_students_location = []
    weight_skill = 0.2
    for student_index in student_indices:
        # print(f"  - {company['com_name'][company_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_location[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_location.append((student_name,weighted_similarity))
    recommended_students_location.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_location

### job size - company size

In [68]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills)

students['lemmatized_jobsize'] = students['preferred_job_size'].apply(lemmatize_skills)
internship['lemmatized_companysize'] = internship['company_size'].apply(lemmatize_skills)

tfi = TfidfVectorizer()
students_jobsize_vectors = tfi.fit_transform(students['lemmatized_jobsize'])
internship_companysize_vectors = tfi.transform(internship['lemmatized_companysize'])

similarity_score_jobsize = cosine_similarity(students_jobsize_vectors,internship_companysize_vectors)

In [69]:
def recommend_jobsize_internship_companysize(student_name_input):
    
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    internship_indices = similarity_score_jobsize[student_index].argsort()[::-1][:19]
    recommended_intenship_jobsize = []
    weight_skill = 0.1
    for internship_index in internship_indices:
        # print(f"  - {company['com_name'][company_index]}")
        internship_name = internship['com_name'][internship_index]
        similarity = similarity_score_jobsize[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_intenship_jobsize.append((internship_name,weighted_similarity))
    recommended_intenship_jobsize.sort(key=lambda x : x[1],reverse=True)    
    return recommended_intenship_jobsize

In [70]:
def recommend_students_jobsize_companysize(internship_name_input):    
    try:
        company_index = internship[internship['com_name'].str.lower() == internship_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{internship_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_jobsize[:, company_index].argsort()[::-1][:10]
    recommended_students_jobsize = []
    weight_skill = 0.1
    for student_index in student_indices:
        # print(f"  - {company['com_name'][company_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_jobsize[student_index, company_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_jobsize.append((student_name,weighted_similarity))
    recommended_students_jobsize.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_jobsize

### category - rank

In [71]:
lemma = WordNetLemmatizer()
def lemmatize_skills(skills_str):
    skills = [skill.strip() for skill in skills_str.split(' ')]
    lemmatized_skills = []
    for skill in skills:
        tokens = word_tokenize(skill.lower())
        lemmas = [lemma.lemmatize(token) for token in tokens] 
        lemmatized_skills.append(" ".join(lemmas))
    return " ".join(lemmatized_skills)

students['lemmatized_category'] = students['Category'].apply(lemmatize_skills)
internship['lemmatized_student_rank'] = internship['student_rank'].apply(lemmatize_skills)

tfi = TfidfVectorizer()
students_category_vectors = tfi.fit_transform(students['lemmatized_category'])
internship_studentrank_vectors = tfi.transform(internship['lemmatized_student_rank'])

similarity_score_category = cosine_similarity(students_category_vectors,internship_studentrank_vectors)

In [72]:
def recommend_internship_rannk_category(student_name_input):
    
    try:
        student_index = students[students['student_name'].str.lower() == student_name_input.lower()].index[0]
    except IndexError:
        print(f"Student with name '{student_name_input}' not found.")
        exit()

    # student_name = students['student_name'][student_index]
    # print(f"Recommendations for {student_name}:")
    internship_indices = similarity_score_category[student_index].argsort()[::-1][:19]
    recommended_internship_category_rank = []
    weight_skill = 0.3
    for internship_index in internship_indices:
        # print(f"  - {company['com_name'][company_index]}")
        internship_name = internship['com_name'][internship_index]
        similarity = similarity_score_category[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_internship_category_rank.append((internship_name,weighted_similarity))
    recommended_internship_category_rank.sort(key=lambda x : x[1],reverse=True)    
    return recommended_internship_category_rank

In [73]:
def recommend_students_rankcategory(internship_name_input):
    try:
        internship_index = internship[internship['com_name'].str.lower() == internship_name_input.lower()].index[0]
    except IndexError:
        print(f"Company with name '{internship_name_input}' not found.")
        return 

    # print(f"Recommendations for {company['com_name'][company_index]}:")
    student_indices = similarity_score_category[:, internship_index].argsort()[::-1][:10]
    recommended_students_category_rank = []
    weight_skill = 0.3
    for student_index in student_indices:
        # print(f"  - {company['com_name'][company_index]}")
        student_name = students['student_name'][student_index]
        similarity = similarity_score_category[student_index, internship_index] # Get the similarity score
        weighted_similarity = similarity * weight_skill
        recommended_students_category_rank.append((student_name,weighted_similarity))
    recommended_students_category_rank.sort(key=lambda x : x[1],reverse=True)    
    return recommended_students_category_rank

### bottom carousel

In [74]:
#recommending companies to students
def merge_and_aggregate_recommendations(skill_recommendations, role_recommendations, location_recommendations,
                                        jobsize_recommendations, category_recommendations):
    

    combined_scores = {}  
    def add_scores_from_list(recommendations, weight):
        if isinstance(recommendations, str): 
            return
        for internship, score in recommendations:
            combined_scores[internship] = combined_scores.get(internship, 0) + score * weight

   
    add_scores_from_list(skill_recommendations, 0.5) 
    add_scores_from_list(role_recommendations, 0.6)   
    add_scores_from_list(location_recommendations, 0.2) 
    add_scores_from_list(jobsize_recommendations, 0.1) 
    add_scores_from_list(category_recommendations, 0.3)
    sorted_recommendations = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations


def recommend_internships_combined(student_name_input):
    skill_recommendations = recommend_internships_skills_prerequisites(student_name_input)
    role_recommendations = recommend_internship_preferredrole_jobtitle(student_name_input) 
    location_recommendations = recommend_internship_location_location(student_name_input)
    jobsize_recommendations = recommend_jobsize_internship_companysize(student_name_input)
    category_recommendations = recommend_internship_rannk_category(student_name_input)
     

    combined_recommendations = merge_and_aggregate_recommendations(
        skill_recommendations, role_recommendations, location_recommendations,
        jobsize_recommendations, category_recommendations
    )

    return combined_recommendations

student_name_input = input("Enter student name: ")
recommendations = recommend_internships_combined(student_name_input)

if recommendations:
    print(f"Combined Recommendations for {student_name_input}:")
    for internship, score in recommendations:
        print(f"  - {internship} (Combined Score: {score:.4f})")
        
# recommendations

Combined Recommendations for shubh gupta:
  - 1mg (Combined Score: 0.4501)
  - upGrad (Combined Score: 0.4329)
  - Ola (Combined Score: 0.4108)
  - Google (Combined Score: 0.3741)
  - Capgemini (Combined Score: 0.3218)
  - Microsoft (Combined Score: 0.2796)
  - Tata Consultancy Services (Combined Score: 0.2717)
  - Cognizant (Combined Score: 0.2627)
  - Paytm (Combined Score: 0.2583)
  - Razorpay (Combined Score: 0.2336)
  - Mindtree (Combined Score: 0.1842)
  - Wipro (Combined Score: 0.1442)


In [75]:
top_internship = recommendations[0]
top_internship

('1mg', 0.45005315001107016)

In [76]:
internship = pd.read_excel('internshipdataset.xlsx')
internship.drop(['Unnamed: 0','com_name.1'],axis=1,inplace=True)

In [77]:
#recommending students to comapnies
def merge_and_aggregate_recommendations(skill_recommendations, role_recommendations, location_recommendations,
                                        jobsize_recommendations, category_recommendations):
    

    combined_scores = {}  
    def add_scores_from_list(recommendations, weight):
        if isinstance(recommendations, str): 
            return
        for students, score in recommendations:
            combined_scores[students] = combined_scores.get(students, 0) + score * weight

   
    add_scores_from_list(skill_recommendations, 0.5) 
    add_scores_from_list(role_recommendations, 0.6)   
    add_scores_from_list(location_recommendations, 0.2) 
    add_scores_from_list(jobsize_recommendations, 0.1) 
    add_scores_from_list(category_recommendations, 0.3)
    sorted_recommendations = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations


def recommend_companies_combined(internship_name_input):
    skill_recommendations = recommend_students_skills(internship_name_input)
    role_recommendations = recommend_students_preferredrole_jobtitle(internship_name_input) 
    location_recommendations = recommend_students_location_location(internship_name_input)
    jobsize_recommendations = recommend_students_jobsize_companysize(internship_name_input)
    category_recommendations = recommend_students_rankcategory(internship_name_input)
    # salary_recommendations = recommend_companies_salary(company_name_input) 

    combined_recommendations = merge_and_aggregate_recommendations(
        skill_recommendations, role_recommendations, location_recommendations,
        jobsize_recommendations, category_recommendations
    )

    return combined_recommendations

internship_name_input = input("Enter company name: ")
recommendations = recommend_companies_combined(internship_name_input)

if recommendations:
    print(f"Combined Recommendations for {internship_name_input}:")
    for students, score in recommendations:
        print(f"  - {students} (Combined Score: {score:.4f})")

Combined Recommendations for google:
  - Shubh Gupta (Combined Score: 0.3741)
  - Ishaan Choudhary (Combined Score: 0.3689)
  - Aditya Chunarkar (Combined Score: 0.3618)
  - Purvi Wasan (Combined Score: 0.3388)
  - Bhoomika Gupta (Combined Score: 0.2866)
  - Devanshu Thakur (Combined Score: 0.2653)
  - Ayush Kirdak (Combined Score: 0.2524)
  - Prakarsh Singh (Combined Score: 0.2213)
  - Arjun Deshpande (Combined Score: 0.2018)
  - Ansh Kirdak (Combined Score: 0.1962)
  - Syed Nousin  (Combined Score: 0.1678)
  - Amitesh Bhattacharya (Combined Score: 0.1109)
  - Diya Runwal (Combined Score: 0.0957)
  - Varun Patney (Combined Score: 0.0900)
  - Darshan Raj (Combined Score: 0.0600)
  - Chaitanya Muley (Combined Score: 0.0328)
  - Devyanshi Saini (Combined Score: 0.0309)
  - Sakshi Negi (Combined Score: 0.0254)
  - Prisha Sharma (Combined Score: 0.0000)
